<a href="https://colab.research.google.com/github/zakio10/nlp-with-transformers/blob/main/ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1章 入門Transformers

In [ ]:
# https://huggingface.co/docs/transformers/installation

In [5]:
!pip install transformers
!pip install sentencepiece # 翻訳用
!pip install sacremoses # 翻訳用
# !pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 4.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=688bf6b7927c776aa90a1dc49e7a687246160bf36eeb2c28ce32bcaed639cf36
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [2]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

In [3]:
# テキスト分類
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [12]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


In [14]:
# 固有表現認識
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,ORG,0.879011,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


In [15]:
# 質問応答
reader = pipeline("question-answering")
question = "What does the customer want?" # 顧客は何を求めていますか？
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


In [21]:
# 要約
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=56, clean_up_tokenization_spaces=True) # min_length has to be more than 56
print(outputs[0]["summary_text"])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I


In [22]:
# outputsの内容確認
print(outputs)

[{'summary_text': ' Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I'}]


In [4]:
# 翻訳
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-27-fce0f829c541> in <module>
#       1 # 翻訳
# ----> 2 translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
#       3 outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
#       4 print(outputs[0]['translation_text'])

# 1 frames
# /usr/local/lib/python3.7/dist-packages/transformers/models/auto/tokenization_auto.py in from_pretrained(cls, pretrained_model_name_or_path, *inputs, **kwargs)
#     613                 else:
#     614                     raise ValueError(
# --> 615                         "This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed "
#     616                         "in order to use this tokenizer."
#     617                     )

# ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von Ihnen zu hören. Aufrichtig, Bumblebee.


In [6]:
# テキスト生成
generator = pipeline("text-generation")
response = "Dear Bumblebee, Iam sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response: \n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response: 
Dear Bumblebee, Iam sorry to hear that your order was mixed up. I also regret we did not have a time to send you a replacement figure for the figure. Instead of the usual Megatron, we wanted a re-designed Optimus Prime. After all, your store would never have had access to the original Transformers toy, which could never have been assembled and packaged in such great haste. As you


# 個人的に追加実験

In [10]:
# 日本語翻訳
# https://huggingface.co/staka/fugumt-en-ja
fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')
outputs = fugu_translator(text, clean_up_tokenization_spaces=True)
print(outputs[0]['translation_text'])

先週 アマゾン様 ドイツのオンラインストアで オプティマス・プライムのアクションフィギュアを注文しました 残念ながら パッケージを開けた時に メガトロンのアクションフィギュアが 送られてきたという恐怖に気付きました ディセプティコンの生涯の敵として 私のジレンマを理解できたら幸いです 問題を解決するために 私が注文したオプティマス・プライムのフィギュアと メガトロンの交換を依頼します この購入に関する私の記録のコピーを封入しました すぐにあなたから連絡が入ります 誠にバンブルビーです


In [15]:
# 日本語翻訳

translator = pipeline("translation", model="facebook/mbart-large-cc25")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/__init__.py:757: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_ro"
  UserWarning,


ro, Bumblebebebeenenenenenenenenenenenenenenenenenenenenenenencomenenencomenencomenencomcomcomenencomcomcomcomcomcomcomcomcomcomcomcomcombebebebebebebebebebebebebecomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomcomenenenenenenenenen
